# Step 1 - Extract the basic frames from the video

In [ ]:
import cv2
import os

# Constants
VIDEO_PATH = "data/tennis_match.mp4"  # Replace with your video file
OUTPUT_FOLDER = "annotated_frames"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)  # Create folder if not exists

# Open video file
cap = cv2.VideoCapture(VIDEO_PATH)
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Stop when video ends

    frame_count += 1
    frame_filename = os.path.join(OUTPUT_FOLDER, f"frame_{frame_count:04d}.jpg")  # Save with zero-padded numbers
    cv2.imwrite(frame_filename, frame)  # Save frame as image
    print(f"Saved {frame_filename}")

cap.release()
print(f"✅ Extraction complete! Frames saved in '{OUTPUT_FOLDER}'")

# Step 2 - Manually run all the frames and annotate players

# Step 3 - Take annotated frames and generate ground truth csv

In [1]:
import cv2
import os
import numpy as np
import pandas as pd

# Constants
FRAME_FOLDER = "annotated_frames"  # Folder containing annotated frames
OUTPUT_CSV = "ground_truth.csv"
LOWER_RED = np.array([0, 50, 50])    # Lower HSV for Red
UPPER_RED = np.array([10, 255, 255])  # Upper HSV for Red
LOWER_YELLOW = np.array([20, 100, 100])  # Lower HSV for Yellow
UPPER_YELLOW = np.array([30, 255, 255])  # Upper HSV for Yellow

# Function to detect players in an annotated frame
def detect_players(image, lower_color, upper_color):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)  # Convert to HSV
    mask = cv2.inRange(hsv, lower_color, upper_color)  # Create a mask for the color
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    bboxes = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w > 10 and h > 10:  # Filter small detections
            bboxes.append((x, y, w, h))
    
    return bboxes

# Process all frames
annotations = []

for filename in sorted(os.listdir(FRAME_FOLDER)):
    if filename.endswith(".jpg"):
        frame_path = os.path.join(FRAME_FOLDER, filename)
        frame = cv2.imread(frame_path)
        frame_id = int(filename.split("_")[1].split(".")[0])  # Extract frame number

        # Detect Player 1 (Red) and Player 2 (Yellow)
        player1_boxes = detect_players(frame, LOWER_RED, UPPER_RED)
        player2_boxes = detect_players(frame, LOWER_YELLOW, UPPER_YELLOW)

        # Store results
        for box in player1_boxes:
            annotations.append([frame_id, 1, *box])  # Player 1
        for box in player2_boxes:
            annotations.append([frame_id, 2, *box])  # Player 2

# Save annotations to CSV
df = pd.DataFrame(annotations, columns=["frame_id", "player_id", "x", "y", "width", "height"])
df.to_csv(OUTPUT_CSV, index=False)

print(f"✅ Ground truth saved to {OUTPUT_CSV}")


✅ Ground truth saved to ground_truth.csv
